In [1]:
import numpy as np
from main_functions_convnets import *

In [71]:
# Now we are going to load some data of cats and dogs to try our algorithm.
# I first import the libraries I need to interact with the directories.
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2

current_dir = '/home/inapifer/Desktop/Deep_Learning'
i = 1000

rute_to_sample_cat = 'Data_cats_dogs/training_set/cats/cat.' + str(i) + '.jpg'


abs_rute_cat = os.path.join(current_dir, rute_to_sample_cat)

print(abs_rute_cat)


cat_matrix = img.imread(abs_rute_cat)


/home/inapifer/Desktop/Deep_Learning/Data_cats_dogs/training_set/cats/cat.1000.jpg
(149, 150, 3)


In [59]:
def convolution(image_input, kernel, stride=1, padding='valid', form='RGB'):
    
    #First we calculate what is the height and width of the input. We consider that the input is given by the shape
    # (height, width, depth)
    height_image, width_image, _ = image_input.shape
    
    #Now we calculate f from the kernel
    f, _, _ = kernel.shape
    
    # The padding is valid if we don't fill the image with 0's on its borders to manipulate dimensions.

    if padding == 'valid':
        p = 0
    
    # Here we calculate the shape of the output and we creatr
    
    height_output, width_output = int((height_image + 2*p -f)/stride + 1), int((width_image + 2*p - f)/stride + 1)
    # I create the output of the convolution after having considered f, the stride and the padding.
    conv_output = np.zeros([height_output, width_output])
    
    
    # Now we go in a for loop going through columns and the going down
    
    for height in range(height_output):
        # Now we go through the columns
        for width in range(width_output):
            
            #The current portion of the convolution will be image[height:(height+f), width:(width+f), :]
            current_portion = image_input[(height*stride):(height*stride+f), (width*stride):(width*stride+f),:]
            #Now I do the convolution operation throughout the whole image.
            conv_output[height, width] = basic_convolution(current_portion, kernel)
                                         
    return conv_output
    

In [70]:
image = np.random.randn(10,9,3)
kernel = np.random.randn(4,4,3)

convolution(image, kernel, stride=2).shape


(4, 3)